## Использование
* Таргеты должны быть в csv-формате (img_id,label,x,y,w,h) - скрипт targets2csv.py
* Для YOLOv5:
** Предикты должны быть в csv-формате (img_id,confidence,label,x,y,w,h) - измененный скрипт val.py
* Для MMDetection:
** Предикты должны быть в формате pkl - для этого установить флаг --out res.pkl для dist-test.sh
** Также нужен файл сплита test.txt для сопоставления id примеров
* Файл .names с перечислением имен классов (одно и

In [89]:
!cat kaspyi/targets.csv

In [1]:
import pandas as pd
import numpy as np
import torch
import pickle

In [36]:
device=torch.device('cuda')

In [78]:
def calc_metrics(correct, targets):    
    TP_cum = torch.cumsum(correct, 0)
    FP_cum = torch.cumsum(torch.logical_not(correct), 0)
    
    GT = targets.shape[0]
    TP = TP_cum[-1, :]
    FP = FP_cum[-1, :]
    FN = torch.maximum(torch.tensor(0), GT - TP)
    
    Precision = TP / (TP + FP + 1e-12)
    Recall = torch.minimum(torch.tensor(1.0), TP / (GT + 1e-12))
    F1 = 2 * Precision * Recall / (Precision + Recall + 1e-12)
    
    precs = TP_cum / (TP_cum + FP_cum + 1e-12)
    recs = TP_cum / (GT + 1e-12)
    
    cummax, _ = torch.cummax(torch.flip(precs, (0,)), 0)
    precs = torch.flip(cummax, (0,))
    
    x = np.linspace(0, 1, 11)
    recs = recs.cpu().numpy()
    precs = precs.cpu().numpy()
    
    interp = np.empty((x.shape[0], precs.shape[1]))
    ap = np.empty((precs.shape[1],))
    for j in range(precs.shape[1]):
        ap[j] = np.trapz(np.interp(x, recs[:, j], precs[:, j]), x)
    
    return TP, FP, FN, GT, Precision, Recall, F1, ap

In [81]:
def pkl_to_df(pkl_path, split_path):
    with open(pkl_path, 'rb') as preds_f:
            data = pickle.load(preds_f)
    with open(split_path, 'r') as test_f:
        images = list(map(lambda line: int(line.rstrip().split('/')[-1][:-4]), test_f.readlines()))
    preds_count = sum(len(l[0]) for l in data)

    df_dict = {'img': np.empty(preds_count, dtype=int),
       'confidence': np.empty(preds_count),
       'label': np.empty(preds_count, dtype=int),
       'x': np.empty(preds_count),
       'y': np.empty(preds_count),
       'w': np.empty(preds_count),
       'h': np.empty(preds_count)}

    start = 0
    end = 0
    for i, image in enumerate(data):
        img_id = images[i]
        for label, preds in enumerate(image):
            end += preds.shape[0]
            df_dict['img'][start:end] = img_id
            df_dict['confidence'][start:end] = preds[:, -1]
            df_dict['label'][start:end] = label
            df_dict['x'][start:end] = preds[:, 0]
            df_dict['y'][start:end] = preds[:, 1]
            df_dict['w'][start:end] = preds[:, 2] - preds[:, 0]
            df_dict['h'][start:end] = preds[:, 3] - preds[:, 1]
            start += preds.shape[0]

    return pd.DataFrame(df_dict)

In [82]:
def get_metrics(targets_csv, width, height, names, preds_csv = None, preds_pkl = None, split = None, center=True):
    if not preds_csv and not preds_pkl:
        raise 'Give .csv or .pkl file'
    if preds_pkl and not split:
        raise 'Give split file'
        
    if preds_csv:
        preds_df = pd.read_csv(preds_path, index_col = 0)
    else:
        preds_df = pkl_to_df(preds_pkl, split)
        
        
    targets_df = pd.read_csv(targets_csv)
    
    targets_df['x'] *= width
    targets_df['w'] *= height

    targets_df['y'] *= width
    targets_df['h'] *= height
    
    if center:
        preds_df['x'] += preds_df['w']/2
        preds_df['y'] += preds_df['h']/2

    preds = torch.as_tensor(preds_df[['x', 'y', 'w', 'h', 'confidence', 'label', 'img']].to_numpy(), device=device)
    targets = torch.as_tensor(targets_df[['x', 'y', 'w', 'h', 'label', 'img']].to_numpy(), device=device)
        
    ious = torch.empty((preds.shape[0]), dtype=torch.double, device=device)
    class_tps = torch.empty((preds.shape[0]), dtype=torch.bool, device=device)

    images = torch.unique(preds[:, -1])
    for image in images:
        idx_preds = preds[:, -1] == image
        img_preds = preds[idx_preds]
        img_targets = targets[targets[:, -1] == image]

        if not img_preds.shape[0] or not img_targets.shape[0]:
            continue

        preds_areas = img_preds[:, 2] * img_preds[:, 3]
        targets_areas = img_targets[:, 2] * img_targets[:, 3]

        xx1_ = img_targets[:, 0].expand(img_preds.shape[0], img_targets.shape[0])
        xx1 = torch.maximum(img_preds[:, 0], xx1_.T)
        yy1_ = img_targets[:, 1].expand(img_preds.shape[0], img_targets.shape[0])
        yy1 = torch.maximum(img_preds[:, 1], yy1_.T)
        xx2_ = (img_targets[:, 2] + img_targets[:, 0]).expand(img_preds.shape[0], img_targets.shape[0])
        xx2 = torch.minimum((img_preds[:, 0] + img_preds[:, 2]), xx2_.T)
        yy2_ = (img_targets[:, 3] + img_targets[:, 1]).expand(img_preds.shape[0], img_targets.shape[0])
        yy2 = torch.minimum(img_preds[:, 1] + img_preds[:, 3], yy2_.T)

        w = torch.maximum(torch.tensor(0.0, device=device), xx2-xx1)
        h = torch.maximum(torch.tensor(0.0, device=device), yy2-yy1)
        inter_areas = w*h

        areas1_ = targets_areas.expand(preds_areas.shape[0], targets_areas.shape[0])
        areas2_ = preds_areas.expand(targets_areas.shape[0], preds_areas.shape[0])
        areas3_ = areas1_ + areas2_.T
        outer_areas = (areas3_- inter_areas.T)
        ious_ = inter_areas.T/outer_areas
        max_ious, argmax_ious = torch.max(ious_, axis=1)

        ious[idx_preds.nonzero().squeeze(1)] = max_ious
        class_tps[idx_preds.nonzero().squeeze(1)] = img_targets[argmax_ious, -2] == img_preds[:, -2]
    preds = torch.hstack([preds, ious.unsqueeze(-1), class_tps.unsqueeze(-1)])
    
    thresh_start=0.001
    thresh_step=0.05
    threshes = [thresh_start] + list(np.around(np.arange(thresh_step, 1.0, thresh_step), 2))

    with open(names, 'r') as names_file:
        class_names = list(map(lambda x: x.rstrip(), names_file.readlines()))
    class_labels = list(range(0, len(class_names)))
    metrics_names = ['TP', 'FP', 'FN', 'GT', 'Precision', 'Recall', 'F1', 'AP50', 'mAP']

    indexes = [threshes, class_names + ['avg', 'localization']]
    multi_index = pd.MultiIndex.from_product(indexes, names=['thresh', 'class'])
    
    metrics_np = np.zeros((len(threshes), 9), dtype=np.float64)
    metrics_detailed_np = np.zeros((len(threshes), len(class_names)+2, 9), dtype=np.float64)

    iou_threshes = np.around((np.arange(0.5, 1.0, 0.05)), 2)

    correct = torch.zeros((preds.shape[0], iou_threshes.shape[0]), dtype=bool, device=device)
    correct_loc = torch.zeros((preds.shape[0], iou_threshes.shape[0]), dtype=bool, device=device)

    for i, iou in enumerate(iou_threshes):
        correct[:, i] = torch.logical_and(preds[:, -1], preds[:, -2] > iou)
        correct_loc[:, i] = preds[:, -2] > iou

    for i, conf_thresh in enumerate(threshes):
        for label, name in zip(class_labels, class_names):
            preds_class_idx = torch.logical_and(preds[:, -5] > conf_thresh, preds[:, -4] == label)
            targets_class = targets[targets[:, -2] == label]
            correct_class = correct[preds_class_idx]
            if not preds_class_idx.any() or not targets_class.shape[0]:
                continue

            TP, FP, FN, GT, Precision, Recall, F1, AP = calc_metrics(correct_class, targets_class)
            metrics_detailed_np[i, label, 0] = TP[0]
            metrics_detailed_np[i, label, 1] = FP[0]
            metrics_detailed_np[i, label, 2] = FN[0]
            metrics_detailed_np[i, label, 3] = GT
            metrics_detailed_np[i, label, 4] = Precision[0]
            metrics_detailed_np[i, label, 5] = Recall[0]
            metrics_detailed_np[i, label, 6] = F1[0]
            metrics_detailed_np[i, label, 7] = AP[0]
            metrics_detailed_np[i, label, 8] = np.mean(AP)

        metrics_detailed_np[i, -2, 0] = np.sum(metrics_detailed_np[i, :-2, 0])
        metrics_detailed_np[i, -2, 1] = np.sum(metrics_detailed_np[i, :-2, 1])
        metrics_detailed_np[i, -2, 2] = np.sum(metrics_detailed_np[i, :-2, 2])
        metrics_detailed_np[i, -2, 3] = np.sum(metrics_detailed_np[i, :-2, 3])
        metrics_detailed_np[i, -2, 4] = metrics_detailed_np[i, -2, 0] / (metrics_detailed_np[i, -2, 0] + metrics_detailed_np[i, -2, 1] + 1e-12)
        metrics_detailed_np[i, -2, 5] = np.minimum(1, metrics_detailed_np[i, -2, 0] / (metrics_detailed_np[i, -2, 3] + 1e-12)) 
        metrics_detailed_np[i, -2, 6] = 2 * metrics_detailed_np[i, -2, 4] * metrics_detailed_np[i, -2, 5] / (metrics_detailed_np[i, -2, 4] + metrics_detailed_np[i, -2, 5] + 1e-12)
        metrics_detailed_np[i, -2, 7] = np.mean(metrics_detailed_np[i, :-2, 7])
        metrics_detailed_np[i, -2, 8] = np.mean(metrics_detailed_np[i, :-2, 8])

        curr_correct_loc = correct_loc[preds[:, -5] > conf_thresh]
        if curr_correct_loc.shape[0]:
            TP, FP, FN, GT, Precision, Recall, F1, AP = calc_metrics(curr_correct_loc, targets)
            metrics_detailed_np[i, -1, 0] = TP[0]
            metrics_detailed_np[i, -1, 1] = FP[0]
            metrics_detailed_np[i, -1, 2] = FN[0]
            metrics_detailed_np[i, -1, 3] = GT
            metrics_detailed_np[i, -1, 4] = Precision[0]
            metrics_detailed_np[i, -1, 5] = Recall[0]
            metrics_detailed_np[i, -1, 6] = F1[0]
            metrics_detailed_np[i, -1, 7] = AP[0]
            metrics_detailed_np[i, -1, 8] = np.mean(AP)

        metrics_np[i, :] = metrics_detailed_np[i, -2, :]
        
    metrics = pd.DataFrame(data=metrics_np, columns = metrics_names,
                      index = threshes)
    metrics_detailed = {}
    for i, thresh in enumerate(threshes):
        cls_df = pd.DataFrame(data=metrics_detailed_np[i, :, :], columns = metrics_names,
                          index = class_names + ['avg', 'loc'])
        metrics_detailed[thresh] = cls_df
    return metrics

In [86]:
pkl_names = [
    'atss',
    'gflv2',
    'reppoints',
            ]

dfs = dict(zip(pkl_names, list(map(lambda name: get_metrics(preds_pkl = 'kaspyi/' + name + '_res.pkl',
                                                            targets_csv = 'kaspyi/targets.csv',
                                                            split = 'kaspyi/test.txt',
                                                            width = 640, height = 640,
                                                            names = 'obj.names'), pkl_names))))

In [87]:
best_F1_stats = pd.DataFrame(columns=metrics_names, index=pkl_names)

for name, df in dfs.items():
    best_F1_stats.loc[name] = df.iloc[df['F1'].argmax()]


In [88]:
best_F1_stats

,TP,FP,FN,GT,Precision,Recall,F1,AP50,mAP
atss,117.0,9.0,8.0,125.0,0.928571,0.936,0.932271,0.948029,0.607576
gflv2,116.0,8.0,9.0,125.0,0.935484,0.928,0.931727,0.959252,0.632595
reppoints,116.0,8.0,9.0,125.0,0.935484,0.928,0.931727,0.974821,0.546028
